In [48]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta
from urllib import parse
import time
from selenium import webdriver
import pyautogui
import time
from os import listdir
from os.path import isfile, join
from tqdm import tqdm

#### 울산 동서발전 주소로 하루마다 크롤링

In [2]:
url = "https://astro.kasi.re.kr/life/pageView/10?useElevation=1&lat=35.572766506061576&lng=129.33345919720028&elevation=-110.51534578549804&output_range=1&date={}&hour=&minute=&second=&address=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C+%EC%A4%91%EA%B5%AC+%EC%A2%85%EA%B0%80%EB%A1%9C+545"

In [3]:
date_list = pd.date_range(start='20180301', end='20210201')

In [5]:
dt_list = date_list.strftime("%Y-%m-%d").tolist()

In [41]:
def makedata(date):
    urldate = url.format(date)
    driver.get(urldate)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contents=soup.select('div.table-responsive > table > tbody > tr')
    columns=soup.select('div.table-responsive > table > thead > tr > th')
    
    columnlist=[]
    dfcontent=[]
    alldfcontents=[]

    for content in contents:
        tds=content.find_all("td")
        for td in tds:
            dfcontent.append(td.text)
        alldfcontents.append(dfcontent)
        dfcontent=[]
        
    for column in columns:
        columnlist.append(column.text)

    df=pd.DataFrame(columns=columnlist, data = alldfcontents)
    
    df['날짜'] = date
    
    df = df[['날짜','시간(시)', '방위각(도 분 초)', '고도(도 분 초)', '적경(시 분 초)', '적위(도 분 초)']]
    
    return df
    
    
    

In [42]:
driver = webdriver.Chrome('../../crawling/chromedriver.exe')

makedata(dt_list[0])

,날짜,시간(시),방위각(도 분 초),고도(도 분 초),적경(시 분 초),적위(도 분 초)
0,2018-03-01,00,341 44 10.55,-61 06 56.08,22 46 00.56,-7 49 53.62
1,2018-03-01,01,013 03 14.64,-61 39 59.25,22 46 10.07,-7 48 56.82
2,2018-03-01,02,040 01 16.71,-56 07 08.78,22 46 19.57,-7 48 00.06
3,2018-03-01,03,058 46 30.90,-46 48 04.78,22 46 29.07,-7 47 03.32
4,2018-03-01,04,071 59 25.33,-35 41 34.84,22 46 38.54,-7 46 06.62
5,2018-03-01,05,082 17 31.52,-23 47 44.61,22 46 47.99,-7 45 09.93
6,2018-03-01,06,091 15 30.49,-11 36 55.70,22 46 57.40,-7 44 13.25
7,2018-03-01,07,099 53 25.52,00 31 36.75,22 47 06.78,-7 43 16.56
8,2018-03-01,08,108 58 56.24,12 20 41.71,22 47 16.13,-7 42 19.86
9,2018-03-01,09,119 20 59.61,23 29 08.94,22 47 25.44,-7 41 23.12


In [43]:
dt_list_small = dt_list[0
                       :2]

In [44]:
dt_list_small

['2018-03-01', '2018-03-02']

In [45]:
total = pd.DataFrame(columns=['날짜','시간(시)', '방위각(도 분 초)', '고도(도 분 초)', '적경(시 분 초)', '적위(도 분 초)'])

In [49]:
for i in tqdm(dt_list):
    
    total = pd.concat([total, makedata(i)], ignore_index=True)
    

100%|██████████████████████████████████████████████████████████████████████████████| 1069/1069 [05:16<00:00,  3.38it/s]


In [52]:
total.isna().sum()

날짜            0
시간(시)         0
방위각(도 분 초)    0
고도(도 분 초)     0
적경(시 분 초)     0
적위(도 분 초)     0
dtype: int64

In [53]:
total.to_csv('울산_태양고도.csv')

#### 당진 위경도로 크롤링

In [56]:
url1 = 'https://astro.kasi.re.kr/life/pageView/10?useElevation=1&lat=37.05127236188373&lng=126.51409076261092&elevation=-111.31808208562506&output_range=1&date={}&hour=&minute=&second=&address=%EC%B6%A9%EC%B2%AD%EB%82%A8%EB%8F%84+%EB%8B%B9%EC%A7%84%EC%8B%9C+%EC%84%9D%EB%AC%B8%EB%A9%B4+%EA%B5%90%EB%A1%9C%EA%B8%B8+30'

In [57]:
def makedata_dangjin(date):
    urldate = url1.format(date)
    driver.get(urldate)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contents=soup.select('div.table-responsive > table > tbody > tr')
    columns=soup.select('div.table-responsive > table > thead > tr > th')
    
    columnlist=[]
    dfcontent=[]
    alldfcontents=[]

    for content in contents:
        tds=content.find_all("td")
        for td in tds:
            dfcontent.append(td.text)
        alldfcontents.append(dfcontent)
        dfcontent=[]
        
    for column in columns:
        columnlist.append(column.text)

    df=pd.DataFrame(columns=columnlist, data = alldfcontents)
    
    df['날짜'] = date
    
    df = df[['날짜','시간(시)', '방위각(도 분 초)', '고도(도 분 초)', '적경(시 분 초)', '적위(도 분 초)']]
    
    return df
    
    
    

In [58]:
total1 = pd.DataFrame(columns=['날짜','시간(시)', '방위각(도 분 초)', '고도(도 분 초)', '적경(시 분 초)', '적위(도 분 초)'])

In [60]:
for i in tqdm(dt_list):
    
    total1 = pd.concat([total1, makedata_dangjin(i)], ignore_index=True)
    

100%|██████████████████████████████████████████████████████████████████████████████| 1069/1069 [06:12<00:00,  2.87it/s]


In [62]:
total1.to_csv('당진_태양고도.csv')

In [64]:
total1

,날짜,시간(시),방위각(도 분 초),고도(도 분 초),적경(시 분 초),적위(도 분 초)
0,2018-03-01,00,341 44 10.55,-61 06 56.08,22 46 00.56,-7 49 53.62
1,2018-03-01,01,013 03 14.64,-61 39 59.25,22 46 10.07,-7 48 56.82
2,2018-03-01,02,040 01 16.71,-56 07 08.78,22 46 19.57,-7 48 00.06
3,2018-03-01,03,058 46 30.90,-46 48 04.78,22 46 29.07,-7 47 03.32
4,2018-03-01,04,071 59 25.33,-35 41 34.84,22 46 38.54,-7 46 06.62
...,...,...,...,...,...,...
27499,2021-02-01,19,258 04 25.97,-12 33 51.12,21 00 53.49,-16 58 50.11
27500,2021-02-01,20,266 37 12.36,-24 24 34.58,21 01 03.69,-16 58 06.72
27501,2021-02-01,21,275 51 13.10,-36 21 28.44,21 01 13.92,-16 57 23.34
27502,2021-02-01,22,287 06 13.44,-48 04 23.44,21 01 24.18,-16 56 39.98


#### 도분초를 도로 변환하는 방법

35도 44분 7.22초를 변환하자

1. 정수자리는 그냥 도를 그대로 가져온다. 35도

2. 소수자리는 다음과 같이 계산한다. (분/60)+(초/3600)

In [110]:
def convertDMS_to_degree(x):
    
    do = x.split(' ')[0]
    min = x.split(' ')[1]
    sec = x.split(' ')[2]
    
    degree = float(do) + (float(min)/60) + (float(sec)/3600)
    
    return degree

In [82]:
t = total1['방위각(도 분 초)'][0]

In [97]:
t

'341 44 10.55'

In [89]:
int(t.split(' ')[1])

44

In [111]:
convertDMS_to_degree(t)

341.7362638888889

In [109]:
df_ex = total1

In [103]:
df_ex['방위각'] = df_ex['방위각(도 분 초)'].apply(lambda x: convertDMS_to_degree(x))

In [104]:
df_ex

,날짜,시간(시),방위각(도 분 초),고도(도 분 초),적경(시 분 초),적위(도 분 초),방위각
0,2018-03-01,00,341 44 10.55,-61 06 56.08,22 46 00.56,-7 49 53.62,341.736264
1,2018-03-01,01,013 03 14.64,-61 39 59.25,22 46 10.07,-7 48 56.82,13.054067
2,2018-03-01,02,040 01 16.71,-56 07 08.78,22 46 19.57,-7 48 00.06,40.021308
3,2018-03-01,03,058 46 30.90,-46 48 04.78,22 46 29.07,-7 47 03.32,58.775250
4,2018-03-01,04,071 59 25.33,-35 41 34.84,22 46 38.54,-7 46 06.62,71.990369
...,...,...,...,...,...,...,...
27499,2021-02-01,19,258 04 25.97,-12 33 51.12,21 00 53.49,-16 58 50.11,258.073881
27500,2021-02-01,20,266 37 12.36,-24 24 34.58,21 01 03.69,-16 58 06.72,266.620100
27501,2021-02-01,21,275 51 13.10,-36 21 28.44,21 01 13.92,-16 57 23.34,275.853639
27502,2021-02-01,22,287 06 13.44,-48 04 23.44,21 01 24.18,-16 56 39.98,287.103733
